# 1 start pods on minikube

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-turtlebot3`.

In [ ]:
export PJ_ROOT="${HOME}/example-turtlebot3"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-turtlebot3
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `example-turtlebot3`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## add auth tokens for the web app of example-turtlebot3

### create new `secrets/auth-tokens.json`

example)
```json
[
    {
        "host": ".*",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "1IqNHfjQsD84mPHvciATObXM3ozfHmX1",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
                }, {
                    "token": "mgMVtijGi6JWX9HT2PFXkZ6xqSdOZVVd",
                    "allowed_paths": ["^/visualizer/positions/$"]
                }
            ],
            "basic_auths": [
              {
                "username": "1JMF6D46",
                "password": "6u5M0bUhfjj7wMdM",
                "allowed_paths": ["/controller/web/", "/visualizer/locus/"]
              }
            ],
            "no_auths": {
                "allowed_paths": ["^.*/static/.*$"]
            }
        }
    }
]
```

#### for macOS

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens|=.+[
  {
    "token": "'$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)'",
    "allowed_paths": ["^/visualizer/positions/$"]
  }
]' | jq '.[0].settings.basic_auths|=.+[
  {
    "username": "'$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8)'",
    "password": "'$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)'",
    "allowed_paths": ["/controller/web/", "/visualizer/locus/"]
  }
]' | jq '.[0].settings.no_auths.allowed_paths|=.+[
  "^.*/static/.*$"
]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json

#### for Ubuntu

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens|=.+[
  {
    "token": "'$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)'",
    "allowed_paths": ["^/visualizer/positions/$"]
  }
]' | jq '.[0].settings.basic_auths|=.+[
  {
    "username": "'$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8)'",
    "password": "'$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16)'",
    "allowed_paths": ["/controller/web/", "/visualizer/locus/"]
  }
]' | jq '.[0].settings.no_auths.allowed_paths|=.+[
  "^.*/static/.*$"
]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json

## change the auth-tokens to kubernetes secrets

### watch `auth` log

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "kubectl logs -f -lapp=auth --all-containers=true"

### delete and re-register auth-tokens to kubernetes secrets

In [ ]:
kubectl delete secret auth-tokens

In [ ]:
kubectl create secret generic auth-tokens --from-file=${CORE_ROOT}/secrets/auth-tokens.json

### confirm the token will be reloaded

**wait a few minutes until the change of secret is detected by Kubernetes.**  
When the new secret is detected, the tokens of auth will be reloaded automatically.

log messages like below will be shown after tokens is reloaded.

```
...
--------
2019/05/21 10:10:27 hosts: [.*]
--------
2019/05/21 10:10:27 bearerTokenAllowedPaths: map[.*:map[cQB5mONfXwP8tHqPQ6kWpRKNzqvbUdfq:[^/orion/.*$ ^/idas/.*$] Tx2b6WD0rYH6uz6Gwe6F2hfaFxp0geg8:[^/visualizer/positions/$]]]
--------
2019/05/21 10:10:27 basicAuthPaths, map[.*:map[/controller/web/:map[xQdM56jY:jKwHUgGGYDYt0UJJ] /visualizer/locus/:map[xQdM56jY:jKwHUgGGYDYt0UJJ]]]
--------
2019/05/21 10:10:27 noAuthPaths, map[.*:[^.*/static/.*$]]
--------
```

## start command proxy service on minikube

In [ ]:
kubectl apply -f controller/cmd-proxy-minikube-service.yaml

In [ ]:
envsubst < controller/cmd-proxy-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=cmd-proxy

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
cmd-proxy-d7cbc6dfb-6hnsp   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-v6rrx   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-zf2fd   1/1       Running   0          46s
```

In [ ]:
kubectl get services -l app=cmd-proxy

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
cmd-proxy   ClusterIP   10.0.172.56   <none>        8888/TCP   1m
```

## start robot visualization service on minikube

In [ ]:
kubectl apply -f controller/robot-visualization-minikube-service.yaml

In [ ]:
export MONGODB_DATABASE="sth_${FIWARE_SERVICE}"
export MONGODB_COLLECTION=$(echo sth_${ROBOT_SERVICEPATH}xffff${ROBOT_ID}xffff${ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
env BEARER_AUTH=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/visualizer/positions/$"))) | .[0].token' -r) envsubst < controller/robot-visualization-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=robot-visualization

example)
```
NAME                                   READY     STATUS    RESTARTS   AGE
robot-visualization-7d69bcdff7-g7p98   1/1       Running   0          13m
robot-visualization-7d69bcdff7-k6mb9   1/1       Running   0          13m
robot-visualization-7d69bcdff7-txsqn   1/1       Running   0          13m
```

In [ ]:
kubectl get services -l app=robot-visualization

example)
```
NAME                  TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
robot-visualization   ClusterIP   10.0.112.72   <none>        8888/TCP   13m
```